This is the inference kernel for BERT pytorch. Check out the amazing Kernel for finetuning BERT by Yuval: https://www.kaggle.com/yuval6967/toxic-bert-plain-vanila

In [1]:
import sys
import os
package_dir_new = "../input/pytorchpretrainedbertnew/pytorch-pretrained-bert/pytorch-pretrained-BERT"
package_dir = "../input/ppbert/pytorch-pretrained-bert/pytorch-pretrained-BERT"
print(os.listdir("../input"))
# print(os.listdir(package_dir))
# print(os.listdir("../input/bert-different-layer"))
# print(os.listdir("../input/b/ertlb094009"))
# print(os.listdir("../input/bertlb093876"))
sys.path.append(package_dir)
# sys.path.append(package_dir_new)
# print(os.listdir("../input"))

['bert2epoch', 'bert-different-layer', 'bert2new', 'bert094048', 'bert094084', 'bert094081', 'bertlb093986', 'bert094066', 'bert-config', 'bertlb094009', 'ppbert', 'jigsaw-unintended-bias-in-toxicity-classification', 'bert094071', 'pytorchpretrainedbertnew', 'bert-pretrained-models', 'bert-2-new', 'bertlb093876']


In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import torch.utils.data
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import warnings
import torch.nn as nn
from pytorch_pretrained_bert import BertTokenizer, BertForSequenceClassification, BertAdam, BertModel
from pytorch_pretrained_bert import BertConfig

warnings.filterwarnings(action='once')
device = torch.device('cuda')

In [3]:
def convert_lines(example, max_seq_length,tokenizer):
    max_seq_length -=2
    all_tokens = []
    longer = 0
    for text in tqdm(example):
        tokens_a = tokenizer.tokenize(text)
        if len(tokens_a)>max_seq_length:
            tokens_a = tokens_a[:max_seq_length]
            longer += 1
        one_token = tokenizer.convert_tokens_to_ids(["[CLS]"]+tokens_a+["[SEP]"])+[0] * (max_seq_length - len(tokens_a))
        all_tokens.append(one_token)
    return np.array(all_tokens)

In [4]:
MAX_SEQUENCE_LENGTH = 220
SEED = 1234
BATCH_SIZE = 32
BERT_MODEL_PATH = '../input/bert-pretrained-models/uncased_l-12_h-768_a-12/uncased_L-12_H-768_A-12/'

np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# bert_config = BertConfig('../input/bert-inference/bert/bert_config.json')
bert_config = BertConfig('../input/bert-config/bert_config.json')
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_PATH, cache_dir=None,do_lower_case=True)

In [5]:
MAX_SEQUENCE_LENGTH = 220
SEED = 1234
EPOCHS = 1
Data_dir="../input/jigsaw-unintended-bias-in-toxicity-classification"
Input_dir = "../input"
WORK_DIR = "../working/"
num_to_load=1000000                         #Train size to match time limit
valid_size= 100000                          #Validation Size
TOXICITY_COLUMN = 'target'

In [6]:
test_df = pd.read_csv("../input/jigsaw-unintended-bias-in-toxicity-classification/test.csv")
test_df['comment_text'] = test_df['comment_text'].astype(str) 
X_test = convert_lines(test_df["comment_text"].fillna("DUMMY_VALUE"), MAX_SEQUENCE_LENGTH, tokenizer)

100%|██████████| 97320/97320 [01:54<00:00, 846.89it/s]


In [7]:
class CustomModel(nn.Module):
    
    def __init__(self, config, aux_number):
        super().__init__()
        # 加载并冻结bert模型参数
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.w1 = torch.nn.Linear(768, 1)
        self.w2 = torch.nn.Linear(768, aux_number)
    def forward(self, seqs, attention_mask=None, labels=None):
        _, seqs = self.bert(seqs, attention_mask=attention_mask, output_all_encoded_layers=False)
        seqs = self.dropout(seqs)
        res = self.w1(seqs)
        res_aux = self.w2(seqs)
        out = torch.cat((res, res_aux), dim=-1)
        
        return out

In [8]:
# model = BertForSequenceClassification(bert_config, num_labels=6)
# model.load_state_dict(torch.load("../input/bert+pre-process/bert_pytorch.bin"))
# model.to(device)
#0.93898
model = BertForSequenceClassification(bert_config, num_labels=7)
model.load_state_dict(torch.load("../input/bert2new/bert_pytorch.bin"))
model.to(device)
#这个是2个out的版本
# model = CustomModel(bert_config, 6)
# model.load_state_dict(torch.load("../input/bert+preprocess/bert_pytorch.bin"))
# model.to(device)
for param in model.parameters():
    param.requires_grad = False
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1)
            )
          )
          (intermediat

In [9]:
test_preds = np.zeros((len(X_test)))
test = torch.utils.data.TensorDataset(torch.tensor(X_test, dtype=torch.long))
test_loader = torch.utils.data.DataLoader(test, batch_size=32, shuffle=False)
tk0 = tqdm(test_loader)
for i, (x_batch,) in enumerate(tk0):
    pred = model(x_batch.to(device), attention_mask=(x_batch > 0).to(device), labels=None)
    test_preds[i * 32:(i + 1) * 32] = pred[:, 0].detach().cpu().squeeze().numpy()

test_pred = torch.sigmoid(torch.tensor(test_preds)).numpy().ravel()

100%|██████████| 3042/3042 [11:55<00:00,  4.25it/s]


In [10]:
# 0.94009
model = BertForSequenceClassification(bert_config, num_labels=7)
model.load_state_dict(torch.load("../input/bertlb094009/results (1)/bert_pytorch.bin"))
model.to(device)
for param in model.parameters():
    param.requires_grad = False
model.eval()

test_preds_ = np.zeros((len(X_test)))
test = torch.utils.data.TensorDataset(torch.tensor(X_test, dtype=torch.long))
test_loader = torch.utils.data.DataLoader(test, batch_size=32, shuffle=False)
tk0 = tqdm(test_loader)
for i, (x_batch,) in enumerate(tk0):
    pred = model(x_batch.to(device), attention_mask=(x_batch > 0).to(device), labels=None)
    test_preds_[i * 32:(i + 1) * 32] = pred[:, 0].detach().cpu().squeeze().numpy()

test_pred_ = torch.sigmoid(torch.tensor(test_preds_)).numpy().ravel()

# # test_pred = (test_pred + test_pred_) / 2

100%|██████████| 3042/3042 [11:54<00:00,  4.26it/s]


In [11]:
#0.93876
model = CustomModel(bert_config, 6)
model.load_state_dict(torch.load("../input/bertlb093876/results/bert_pytorch.bin"))
model.to(device)
for param in model.parameters():
    param.requires_grad = False
model.eval()

test_preds_2 = np.zeros((len(X_test)))
test = torch.utils.data.TensorDataset(torch.tensor(X_test, dtype=torch.long))
test_loader = torch.utils.data.DataLoader(test, batch_size=32, shuffle=False)
tk0 = tqdm(test_loader)
for i, (x_batch,) in enumerate(tk0):
    pred = model(x_batch.to(device), attention_mask=(x_batch > 0).to(device), labels=None)
    test_preds_2[i * 32:(i + 1) * 32] = pred[:, 0].detach().cpu().squeeze().numpy()

test_pred_2 = torch.sigmoid(torch.tensor(test_preds_2)).numpy().ravel()


100%|██████████| 3042/3042 [11:54<00:00,  4.26it/s]


In [12]:
# sys.path.append(package_dir_new)
# #0.93986
# from pytorch_pretrained_bert import BertTokenizer, BertForSequenceClassification, BertAdam, BertModel
# from pytorch_pretrained_bert import BertConfig
# model = BertForSequenceClassification(bert_config, num_labels=6)
# model.load_state_dict(torch.load("../input/bert+pre-process/bert_pytorch.bin"))
# model.to(device)
# for param in model.parameters():
#     param.requires_grad = False
# model.eval()

# test_preds_3 = np.zeros((len(X_test)))
# test = torch.utils.data.TensorDataset(torch.tensor(X_test, dtype=torch.long))
# test_loader = torch.utils.data.DataLoader(test, batch_size=32, shuffle=False)
# tk0 = tqdm(test_loader)
# for i, (x_batch,) in enumerate(tk0):
#     pred = model(x_batch.to(device), attention_mask=(x_batch > 0).to(device), labels=None)
#     test_preds_3[i * 32:(i + 1) * 32] = pred[:, 0].detach().cpu().squeeze().numpy()

# test_pred_3 = torch.sigmoid(torch.tensor(test_preds_3)).numpy().ravel()

In [13]:
# 

model = BertForSequenceClassification(bert_config, num_labels=7)
model.load_state_dict(torch.load("../input/bert094048/results/bert_pytorch.bin"))
model.to(device)
for param in model.parameters():
    param.requires_grad = False
model.eval()

test_preds_4 = np.zeros((len(X_test)))
test = torch.utils.data.TensorDataset(torch.tensor(X_test, dtype=torch.long))
test_loader = torch.utils.data.DataLoader(test, batch_size=32, shuffle=False)
tk0 = tqdm(test_loader)
for i, (x_batch,) in enumerate(tk0):
    pred = model(x_batch.to(device), attention_mask=(x_batch > 0).to(device), labels=None)
    test_preds_4[i * 32:(i + 1) * 32] = pred[:, 0].detach().cpu().squeeze().numpy()

test_pred_4 = torch.sigmoid(torch.tensor(test_preds_4)).numpy().ravel()

# test_pred = (test_pred + test_pred_) / 2

100%|██████████| 3042/3042 [11:54<00:00,  4.26it/s]


In [14]:
class CustomModel_diff(nn.Module):
    
    def __init__(self, config, aux_number):
        super().__init__()
        # 加载并冻结bert模型参数
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
#         self.w1 = torch.nn.Linear(768, 1)
        self.w1 = torch.nn.Linear(768 * 2, aux_number)
    def forward(self, seqs, attention_mask=None, labels=None):
        seqs_list, seqs = self.bert(seqs, attention_mask=attention_mask, output_all_encoded_layers=True)
        seqs_ = seqs_list[-1]
        bert_seqs = seqs_
        avg_pool = torch.mean(bert_seqs, 1)
        # global max pooling
        max_pool, _ = torch.max(bert_seqs, 1)
        
        #[batch, dim_bert * 2]
        h_conc = torch.cat((max_pool, avg_pool), 1)
        out = self.w1(h_conc)
        
#         seqs = self.dropout(seqs)
#         res = self.w1(seqs)
#         res_aux = self.w2(seqs)
#         out = torch.cat((res, res_aux), dim=-1)
        
        return out

In [15]:

# # model = CustomModel('../working', bert_config, 7)
# model = CustomModel_diff(bert_config, 7)
# model.load_state_dict(torch.load("../input/bert-different-layer/bert_pytorch.bin"))
# model.to(device)
# for param in model.parameters():
#     param.requires_grad = False
# model.eval()

# test_preds_5 = np.zeros((len(X_test)))
# test = torch.utils.data.TensorDataset(torch.tensor(X_test, dtype=torch.long))
# test_loader = torch.utils.data.DataLoader(test, batch_size=32, shuffle=False)
# tk0 = tqdm(test_loader)
# for i, (x_batch,) in enumerate(tk0):
#     pred = model(x_batch.to(device), attention_mask=(x_batch > 0).to(device), labels=None)
#     test_preds_5[i * 32:(i + 1) * 32] = pred[:, 0].detach().cpu().squeeze().numpy()

# test_pred_5 = torch.sigmoid(torch.tensor(test_preds_5)).numpy().ravel()


In [16]:
# 0.94066

model = BertForSequenceClassification(bert_config, num_labels=7)
model.load_state_dict(torch.load("../input/bert094066/results (1)/bert_pytorch.bin"))
model.to(device)
for param in model.parameters():
    param.requires_grad = False
model.eval()

test_preds_5 = np.zeros((len(X_test)))
test = torch.utils.data.TensorDataset(torch.tensor(X_test, dtype=torch.long))
test_loader = torch.utils.data.DataLoader(test, batch_size=32, shuffle=False)
tk0 = tqdm(test_loader)
for i, (x_batch,) in enumerate(tk0):
    pred = model(x_batch.to(device), attention_mask=(x_batch > 0).to(device), labels=None)
    test_preds_5[i * 32:(i + 1) * 32] = pred[:, 0].detach().cpu().squeeze().numpy()

test_pred_5 = torch.sigmoid(torch.tensor(test_preds_5)).numpy().ravel()

# test_pred = (test_pred + test_pred_) / 2

100%|██████████| 3042/3042 [11:54<00:00,  4.26it/s]


In [17]:
# 0.94084

model = BertForSequenceClassification(bert_config, num_labels=7)
model.load_state_dict(torch.load("../input/bert094084/results/bert_pytorch.bin"))
model.to(device)
for param in model.parameters():
    param.requires_grad = False
model.eval()

test_preds_6 = np.zeros((len(X_test)))
test = torch.utils.data.TensorDataset(torch.tensor(X_test, dtype=torch.long))
test_loader = torch.utils.data.DataLoader(test, batch_size=32, shuffle=False)
tk0 = tqdm(test_loader)
for i, (x_batch,) in enumerate(tk0):
    pred = model(x_batch.to(device), attention_mask=(x_batch > 0).to(device), labels=None)
    test_preds_6[i * 32:(i + 1) * 32] = pred[:, 0].detach().cpu().squeeze().numpy()

test_pred_6 = torch.sigmoid(torch.tensor(test_preds_6)).numpy().ravel()

# test_pred = (test_pred + test_pred_) / 2

100%|██████████| 3042/3042 [11:54<00:00,  4.26it/s]


In [18]:
# # 0.94081

# model = BertForSequenceClassification(bert_config, num_labels=7)
# model.load_state_dict(torch.load("../input/bert094081/results/bert_pytorch.bin"))
# model.to(device)
# for param in model.parameters():
#     param.requires_grad = False
# model.eval()

# test_preds_7 = np.zeros((len(X_test)))
# test = torch.utils.data.TensorDataset(torch.tensor(X_test, dtype=torch.long))
# test_loader = torch.utils.data.DataLoader(test, batch_size=32, shuffle=False)
# tk0 = tqdm(test_loader)
# for i, (x_batch,) in enumerate(tk0):
#     pred = model(x_batch.to(device), attention_mask=(x_batch > 0).to(device), labels=None)
#     test_preds_7[i * 32:(i + 1) * 32] = pred[:, 0].detach().cpu().squeeze().numpy()

# test_pred_7 = torch.sigmoid(torch.tensor(test_preds_7)).numpy().ravel()

# # test_pred = (test_pred + test_pred_) / 2

In [19]:
# 0.94071

model = BertForSequenceClassification(bert_config, num_labels=7)
model.load_state_dict(torch.load("../input/bert094071/results (1)/bert_pytorch.bin"))
model.to(device)
for param in model.parameters():
    param.requires_grad = False
model.eval()

test_preds_8 = np.zeros((len(X_test)))
test = torch.utils.data.TensorDataset(torch.tensor(X_test, dtype=torch.long))
test_loader = torch.utils.data.DataLoader(test, batch_size=32, shuffle=False)
tk0 = tqdm(test_loader)
for i, (x_batch,) in enumerate(tk0):
    pred = model(x_batch.to(device), attention_mask=(x_batch > 0).to(device), labels=None)
    test_preds_8[i * 32:(i + 1) * 32] = pred[:, 0].detach().cpu().squeeze().numpy()

test_pred_8 = torch.sigmoid(torch.tensor(test_preds_8)).numpy().ravel()

# test_pred = (test_pred + test_pred_) / 2

100%|██████████| 3042/3042 [11:54<00:00,  4.26it/s]


In [20]:
submission = pd.DataFrame.from_dict({
    'id': test_df['id'],
    'prediction': np.average([test_pred, test_pred_,test_pred_2,test_pred_4, test_pred_5, test_pred_6, test_pred_8],axis=0,weights=[0.1,0.05,0.1,0.1,0.15,0.25,0.15])
    # np.average([test_pred, test_pred_, test_pred_4, test_pred_5, test_pred_6, test_pred_7 ], axis=0) 0.94135
    #np.average([test_pred, test_pred_, test_pred_4, test_pred_5, test_pred_6, test_pred_7 ], axis=0, weights=[0.1,0.1,0.1,0.2,0.3,0.2]
    #np.average([test_pred_,test_pred_6, test_pred_5], axis=0, weights = [0.3, 0.5, 0.2]) 0.94110
    #np.average([test_pred_,test_pred_6, test_pred], axis=0, weights = [0.3, 0.5, 0.2]) 0.94139
    #np.average([test_pred_5,test_pred_4, test_pred_], axis=0, weights = [0.5, 0.3, 0.2]) 0.94092
    #np.average([test_pred_,test_pred_4, test_pred], axis=0, weights = [0.3, 0.5, 0.2]) 0.94123
#     np.average([test_pred_,test_pred_4], axis=0, weights = [0.3, 0.7]) 0.94085 [0.4, 0.6] 0.94098
    #np.average([test_pred, test_pred_,test_pred_2], axis=0, weights = [0.2, 0.6, 0.2]) 0.94092
    #np.average([test_pred, test_pred_,test_pred_2], axis=0, weights = [0.3, 0.5, 0.2]) 0.94052
    #np.mean([test_pred, test_pred_], axis=0)
})
submission.to_csv('submission.csv', index=False)

In [21]:
submission.head()

,id,prediction
0,7000000,0.000225
1,7000001,0.000049
2,7000002,0.001499
3,7000003,0.000284
4,7000004,0.990843
